# Iris 예제를 다양한 머신러닝 방법으로 구현하기

# 1. Regression
- Tensorflow Keras로 구현

In [1]:
# 필요한 module import



import numpy as np
import pandas as pd
from sklearn import datasets

In [12]:
# Raw Data Loading



iris = datasets.load_iris()

# 어떤 데이터들이 있는지 살펴보기
# print(type(iris))   # <class 'sklearn.utils._bunch.Bunch'>
# print(iris['DESCR'])
print(iris.keys()) # dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
                   # 'feature_names' : 컬럼명

# DataFrame으로 변환하기
df = pd.DataFrame(iris['data'],
                  columns=iris['feature_names'])
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
df['target'] = iris['target']   # target을 추가
display(df.info())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   target        150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


None

In [7]:
# Data Preprocessing
x_data = df.drop('target', axis=1, inplace=False).values
t_data = df['target'].values

In [10]:
# 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

In [11]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split

x_data_train_norm, x_data_test_norm, t_data_train, t_data_test = \
train_test_split(x_data_norm,
                 t_data,
                 test_size=0.3)

In [14]:
# Model 구현
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam


model = Sequential()

model.add(Flatten(input_shape=(4,)))    # iris['feature_names'] 4개 -> 독립 변수의 개수

model.add(Dense(units=3,                # iris['target'] 3개 -> 종속변수가 몇개로 나누어 지는지 0,1,2
                activation='softmax'))  # 다중분류 : 'softmax' 사용

model.compile(optimizer=Adam(learning_rate=1e-1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',
                   patience=6)



history = model.fit(x_data_train_norm,
                    t_data_train,
                    epochs=500,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[es])

print(model.evaluate(x_data_test_norm, t_data_test))

Epoch 1/500
3/3 [==============================] - 1s 104ms/step - loss: 1.1625 - accuracy: 0.3095 - val_loss: 0.9674 - val_accuracy: 0.4762
Epoch 2/500
3/3 [==============================] - 0s 18ms/step - loss: 0.9358 - accuracy: 0.5595 - val_loss: 0.8606 - val_accuracy: 0.7143
Epoch 3/500
3/3 [==============================] - 0s 19ms/step - loss: 0.8528 - accuracy: 0.6905 - val_loss: 0.7491 - val_accuracy: 0.7143
Epoch 4/500
3/3 [==============================] - 0s 18ms/step - loss: 0.7442 - accuracy: 0.6905 - val_loss: 0.6500 - val_accuracy: 0.7143
Epoch 5/500
3/3 [==============================] - 0s 18ms/step - loss: 0.6682 - accuracy: 0.7143 - val_loss: 0.6029 - val_accuracy: 0.9524
Epoch 6/500
3/3 [==============================] - 0s 18ms/step - loss: 0.6228 - accuracy: 0.9286 - val_loss: 0.5511 - val_accuracy: 0.9524
Epoch 7/500
3/3 [==============================] - 0s 17ms/step - loss: 0.5744 - accuracy: 0.7738 - val_loss: 0.5052 - val_accuracy: 0.7143
Epoch 8/500
3/3 [==